# Librerías y carga de datos

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('uploads/houseprices.csv')
df.shape

# Columnas principales del dataset

In [ ]:
df.columns

# 20 Primeros registros del DataSet

In [ ]:
df.head(20)

# Características Estadísticas del Dataset

In [ ]:
# Resumen estadístico de las variables numéricas
df.describe()

# Cantidad de nulos y sus tipos

In [ ]:
conteo_nulos = df.isna().sum().sort_values(ascending=False)
conteo_nulos = conteo_nulos[conteo_nulos > 0]

df_nulos = pd.DataFrame({
    'Nulos': conteo_nulos,
    'Tipo': df[conteo_nulos.index].dtypes
})

df_nulos

# Top 15 Nulos con porcentaje y gráfico

In [ ]:
conteo_nulos = df.isna().sum().sort_values(ascending=False)
pct_nulos = (df.isna().mean()*100).sort_values(ascending=False)

display(pd.DataFrame({"nulos": conteo_nulos, "%": pct_nulos}).head(15))

plt.figure(figsize=(10,6))
sns.barplot(x=conteo_nulos.head(15).values, y=conteo_nulos.head(15).index, palette='Reds_r')
plt.title("Top 15 columnas con más nulos", fontsize=14, fontweight='bold')
plt.xlabel("Cantidad de nulos")
plt.tight_layout()
plt.show()

# PASO 1: Rellenar TODOS los valores nulos

In [ ]:
df_listo = df.copy()

# Contar nulos por tipo ANTES de rellenar
nulos_numericos = 0
nulos_categoricos = 0

for col in df_listo.columns:
    if df_listo[col].isnull().any():
        if df_listo[col].dtype in ['int64', 'float64']:
            nulos_numericos += df_listo[col].isnull().sum()
            df_listo[col] = df_listo[col].fillna(0)
        else:
            nulos_categoricos += df_listo[col].isnull().sum()
            df_listo[col] = df_listo[col].fillna('None')

total_rellenados = nulos_numericos + nulos_categoricos

print(f"✅ {total_rellenados:,} valores nulos rellenados:")
print(f"   • Numéricos: {nulos_numericos:,} → 0")
print(f"   • Categóricos: {nulos_categoricos:,} → 'None'")
print(f"\nNulos restantes: {df_listo.isna().sum().sum()}")

# Gráfico: Distribución de nulos rellenados

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
categorias = ['Numéricos', 'Categóricos']
valores = [nulos_numericos, nulos_categoricos]
colores = ['#60a5fa', '#4ade80']

wedges, texts, autotexts = ax.pie(valores, labels=categorias, autopct='%1.1f%%',
                                    colors=colores, startangle=90,
                                    textprops={'fontsize': 12, 'fontweight': 'bold'})
ax.set_title(f'{total_rellenados:,} Nulos Rellenados por Tipo', fontsize=14, fontweight='bold')

for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontsize(14)

plt.tight_layout()
plt.show()

# Análisis de correlaciones con SalePrice

In [ ]:
df_numerico = df_listo.select_dtypes(include=[np.number])
correlaciones = df_numerico.corr()['SalePrice'].sort_values(ascending=False)

print("Top 15 características correlacionadas con SalePrice:")
print("=" * 60)
display(correlaciones.drop('SalePrice').head(15))

# Gráfico: Correlaciones TOP 15

In [ ]:
top_correlaciones = correlaciones.drop('SalePrice').head(15)

plt.figure(figsize=(10,7))
colores_barra = ['#4ade80' if x > 0.5 else '#60a5fa' if x > 0 else 'red' for x in top_correlaciones.values]
plt.barh(top_correlaciones.index, top_correlaciones.values, color=colores_barra, alpha=0.8, edgecolor='black')
plt.xlabel('Correlación con SalePrice', fontsize=12, fontweight='bold')
plt.title('Top 15 Features por Correlación', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)

for i, (idx, val) in enumerate(top_correlaciones.items()):
    plt.text(val + 0.01, i, f'{val:.3f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

# PASO 2: Seleccionar TOP 10 características

In [ ]:
# Basado en el análisis de correlaciones, seleccionamos TOP 10
top_10_caracteristicas = ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea',
                          'TotalBsmtSF', '1stFlrSF', 'FullBath', 'YearBuilt',
                          'YearRemodAdd', 'TotRmsAbvGrd']

columnas_antes = len(df_listo.columns)
df_listo = df_listo[top_10_caracteristicas + ['SalePrice']]

print(f"✅ Dataset reducido de {columnas_antes} a {len(df_listo.columns)} columnas")
print(f"\nCaracterísticas seleccionadas: {', '.join(top_10_caracteristicas)}")

# PASO 3: Eliminar duplicados

In [ ]:
filas_antes = len(df_listo)
df_listo = df_listo.drop_duplicates()
duplicados_eliminados = filas_antes - len(df_listo)

if duplicados_eliminados > 0:
    print(f"✅ {duplicados_eliminados} filas duplicadas eliminadas")
else:
    print("✅ No se encontraron duplicados")

# PASO 4: Remover outliers extremos (percentil 99.5)

In [ ]:
# Guardar datos ANTES para gráfico
area_antes = df_listo['GrLivArea'].copy()
precio_antes = df_listo['SalePrice'].copy()
filas_antes = len(df_listo)

# Remover outliers
umbral = df_listo['GrLivArea'].quantile(0.995)
df_listo = df_listo[df_listo['GrLivArea'] <= umbral]
outliers_eliminados = filas_antes - len(df_listo)

print(f"✅ {outliers_eliminados} outliers eliminados (áreas >{umbral:.0f} pies²)")

# Gráfico: Outliers ANTES vs DESPUÉS

In [ ]:
fig, ejes = plt.subplots(1, 2, figsize=(16, 6))

# ANTES
ejes[0].scatter(area_antes, precio_antes, alpha=0.6, s=30, color='#ef4444', edgecolors='black', linewidth=0.5)
ejes[0].set_xlabel('GrLivArea (pies²)', fontsize=11, fontweight='bold')
ejes[0].set_ylabel('SalePrice ($)', fontsize=11, fontweight='bold')
ejes[0].set_title(f'ANTES: {len(area_antes):,} casas (con outliers)', fontsize=13, fontweight='bold')
ejes[0].grid(alpha=0.3)

# DESPUÉS
ejes[1].scatter(df_listo['GrLivArea'], df_listo['SalePrice'], alpha=0.6, s=30, color='#4ade80', edgecolors='black', linewidth=0.5)
ejes[1].set_xlabel('GrLivArea (pies²)', fontsize=11, fontweight='bold')
ejes[1].set_ylabel('SalePrice ($)', fontsize=11, fontweight='bold')
ejes[1].set_title(f'DESPUÉS: {len(df_listo):,} casas (sin outliers)', fontsize=13, fontweight='bold')
ejes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# PASO 5: Transformación logarítmica de SalePrice

In [ ]:
precio_original = df_listo['SalePrice'].copy()
df_listo['SalePrice_log'] = np.log1p(df_listo['SalePrice'])

print(f"✅ SalePrice transformado a escala logarítmica")
print(f"   Rango original: ${precio_original.min():,.0f} - ${precio_original.max():,.0f}")
print(f"   Rango log: {df_listo['SalePrice_log'].min():.2f} - {df_listo['SalePrice_log'].max():.2f}")

# Gráfico: Distribución ORIGINAL vs LOGARÍTMICA

In [ ]:
fig, ejes = plt.subplots(1, 2, figsize=(16, 6))

# ORIGINAL
ejes[0].hist(precio_original, bins=50, color='#60a5fa', alpha=0.7, edgecolor='black')
ejes[0].axvline(precio_original.mean(), color='red', linestyle='--', linewidth=2, label=f'Media: ${precio_original.mean():,.0f}')
ejes[0].set_xlabel('SalePrice ($)', fontsize=11, fontweight='bold')
ejes[0].set_ylabel('Frecuencia', fontsize=11, fontweight='bold')
ejes[0].set_title('Distribución ORIGINAL (Asimétrica)', fontsize=13, fontweight='bold')
ejes[0].legend()
ejes[0].grid(alpha=0.3)

# LOG
ejes[1].hist(df_listo['SalePrice_log'], bins=50, color='#4ade80', alpha=0.7, edgecolor='black')
ejes[1].axvline(df_listo['SalePrice_log'].mean(), color='red', linestyle='--', linewidth=2, label=f'Media: {df_listo["SalePrice_log"].mean():.2f}')
ejes[1].set_xlabel('Log(SalePrice)', fontsize=11, fontweight='bold')
ejes[1].set_ylabel('Frecuencia', fontsize=11, fontweight='bold')
ejes[1].set_title('Distribución LOGARÍTMICA (Normalizada)', fontsize=13, fontweight='bold')
ejes[1].legend()
ejes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Resumen del dataset limpio

In [ ]:
print("=" * 80)
print("📊 RESUMEN DEL DATASET LIMPIO")
print("=" * 80)
print(f"Filas: {len(df_listo):,}")
print(f"Columnas: {len(df_listo.columns)}")
print(f"Nulos: {df_listo.isna().sum().sum()}")
print("=" * 80)

# Preparar datos para entrenamiento (X, y)

In [ ]:
X = df_listo[top_10_caracteristicas]
y = df_listo['SalePrice_log']

print(f"Shape de X: {X.shape}")
print(f"Shape de y: {y.shape}")

# Dividir en Train/Test (80/20)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Train: {len(X_train)} muestras ({len(X_train)/len(X)*100:.1f}%)")
print(f"Test:  {len(X_test)} muestras ({len(X_test)/len(X)*100:.1f}%)")

# Entrenar modelo XGBoost con optimización de hiperparámetros

In [ ]:
print("🤖 Iniciando RandomizedSearchCV...")
print("   • 20 combinaciones aleatorias")
print("   • 5-fold cross-validation")
print("\n⏳ Esto puede tomar varios minutos...\n")

dist_parametros = {
    'n_estimators': [500, 1000, 1500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'gamma': [0, 0.1],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [0.5, 1, 2]
}

modelo_xgb = xgb.XGBRegressor(random_state=42, eval_metric='rmse')

busqueda_aleatoria = RandomizedSearchCV(
    estimator=modelo_xgb,
    param_distributions=dist_parametros,
    n_iter=20,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

busqueda_aleatoria.fit(X_train, y_train)

modelo = busqueda_aleatoria.best_estimator_
mejores_params = busqueda_aleatoria.best_params_

print("\n" + "=" * 80)
print("✅ OPTIMIZACIÓN COMPLETADA")
print("=" * 80)
print("\nMejores hiperparámetros:")
for param, valor in mejores_params.items():
    print(f"  • {param}: {valor}")
print(f"\nMejor RMSE en CV: {-busqueda_aleatoria.best_score_:.4f}")

# Evaluar modelo en conjunto de prueba

In [ ]:
# Predicciones
y_pred = modelo.predict(X_test)

# Convertir a escala original
y_test_orig = np.expm1(y_test)
y_pred_orig = np.expm1(y_pred)

# Métricas
r2 = r2_score(y_test_orig, y_pred_orig)
rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
mae = mean_absolute_error(y_test_orig, y_pred_orig)

print("=" * 80)
print("📊 EVALUACIÓN DEL MODELO")
print("=" * 80)
print(f"R² Score: {r2:.4f} ({r2*100:.2f}% de varianza explicada)")
print(f"RMSE: ${rmse:,.0f}")
print(f"MAE: ${mae:,.0f}")
print(f"\n💡 El modelo predice con un error promedio de ${mae:,.0f}")

# Gráfico: Predicciones vs Valores Reales

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(y_test_orig, y_pred_orig, alpha=0.6, s=50, color='steelblue', edgecolors='black', linewidth=0.5)

valor_min = min(y_test_orig.min(), y_pred_orig.min())
valor_max = max(y_test_orig.max(), y_pred_orig.max())
plt.plot([valor_min, valor_max], [valor_min, valor_max], 'r--', lw=3, label='Predicción Perfecta')

plt.xlabel('Precio Real ($)', fontsize=12, fontweight='bold')
plt.ylabel('Precio Predicho ($)', fontsize=12, fontweight='bold')
plt.title('Predicciones vs Valores Reales', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)

plt.text(0.05, 0.95, f'R² = {r2:.4f}', transform=plt.gca().transAxes,
         fontsize=14, fontweight='bold', verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

# Gráfico: Residuos

In [ ]:
residuos = y_test_orig - y_pred_orig

fig, ejes = plt.subplots(1, 2, figsize=(16, 6))

# Residuos vs Predicciones
ejes[0].scatter(y_pred_orig, residuos, alpha=0.6, s=40, color='coral', edgecolors='black', linewidth=0.5)
ejes[0].axhline(0, color='red', linestyle='--', linewidth=2)
ejes[0].set_xlabel('Precio Predicho ($)', fontsize=11, fontweight='bold')
ejes[0].set_ylabel('Residuo ($)', fontsize=11, fontweight='bold')
ejes[0].set_title('Residuos vs Predicciones', fontsize=13, fontweight='bold')
ejes[0].grid(alpha=0.3)

# Histograma de residuos
ejes[1].hist(residuos, bins=50, color='coral', alpha=0.7, edgecolor='black')
ejes[1].axvline(0, color='red', linestyle='--', linewidth=2)
ejes[1].set_xlabel('Residuo ($)', fontsize=11, fontweight='bold')
ejes[1].set_ylabel('Frecuencia', fontsize=11, fontweight='bold')
ejes[1].set_title('Distribución de Residuos', fontsize=13, fontweight='bold')
ejes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Media de residuos: ${residuos.mean():,.0f} (idealmente 0)")
print(f"Desviación estándar: ${residuos.std():,.0f}")

# Importancia de características

In [ ]:
importancia_caracteristicas = pd.DataFrame({
    'caracteristica': top_10_caracteristicas,
    'importancia': modelo.feature_importances_
}).sort_values('importancia', ascending=False)

display(importancia_caracteristicas)

# Gráfico: Importancia de características

In [ ]:
plt.figure(figsize=(12, 7))
colores_imp = ['#4ade80' if imp > importancia_caracteristicas['importancia'].mean() else '#60a5fa'
               for imp in importancia_caracteristicas['importancia']]
barras = plt.barh(importancia_caracteristicas['caracteristica'], importancia_caracteristicas['importancia'],
                  color=colores_imp, alpha=0.8, edgecolor='black', linewidth=1.5)
plt.xlabel('Importancia', fontsize=12, fontweight='bold')
plt.title('Importancia de Características en XGBoost', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)

for barra, imp in zip(barras, importancia_caracteristicas['importancia']):
    plt.text(barra.get_width() + 0.005, barra.get_y() + barra.get_height()/2,
             f'{imp*100:.1f}%', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

# 🎲 Generar datos aleatorios para predicciones

In [ ]:
def generar_casa_aleatoria():
    """Genera una casa con características aleatorias realistas"""
    casa = {}
    for caract in top_10_caracteristicas:
        if caract == 'OverallQual':
            casa[caract] = np.random.randint(3, 11)
        elif caract == 'GrLivArea':
            casa[caract] = np.random.randint(600, 4000)
        elif caract == 'GarageCars':
            casa[caract] = np.random.randint(0, 4)
        elif caract == 'GarageArea':
            casa[caract] = casa.get('GarageCars', 2) * 250
        elif caract == 'TotalBsmtSF':
            casa[caract] = np.random.randint(0, 2500)
        elif caract == '1stFlrSF':
            casa[caract] = np.random.randint(400, 2500)
        elif caract == 'FullBath':
            casa[caract] = np.random.randint(1, 4)
        elif caract == 'YearBuilt':
            casa[caract] = np.random.randint(1950, 2024)
        elif caract == 'YearRemodAdd':
            casa[caract] = np.random.randint(casa.get('YearBuilt', 1950), 2024)
        elif caract == 'TotRmsAbvGrd':
            casa[caract] = np.random.randint(3, 12)
        else:
            casa[caract] = np.random.randint(int(X[caract].min()), int(X[caract].max()))
    return casa

print("✅ Función generar_casa_aleatoria() creada")

# Generar 10 casas aleatorias y predecir

In [ ]:
casas = [generar_casa_aleatoria() for _ in range(10)]
df_casas = pd.DataFrame(casas)

predicciones_log = modelo.predict(df_casas)
predicciones = np.expm1(predicciones_log)

df_casas['Precio_Predicho'] = predicciones
df_casas['Precio_Predicho'] = df_casas['Precio_Predicho'].apply(lambda x: f"${x:,.0f}")

display(df_casas[['OverallQual', 'GrLivArea', 'GarageCars', 'YearBuilt', 'FullBath', 'Precio_Predicho']])

# Casa individual aleatoria

In [ ]:
casa = generar_casa_aleatoria()

print("🏡 Casa aleatoria generada:")
print("=" * 60)
for clave, valor in casa.items():
    print(f"{clave:20s}: {valor}")

X_casa = pd.DataFrame([casa])
precio_log = modelo.predict(X_casa)[0]
precio = np.expm1(precio_log)

print("\n" + "=" * 60)
print(f"💰 PRECIO PREDICHO: ${precio:,.0f}")
print("=" * 60)

# ✅ Pipeline Completado

In [ ]:
print("="*80)
print("✅ PIPELINE COMPLETADO EXITOSAMENTE")
print("="*80)
print(f"\n📊 Modelo entrenado con R² = {r2:.4f}")
print(f"📉 Error promedio: ${mae:,.0f}")
print(f"🎯 Listo para hacer predicciones!")